In [1]:
# Ignore warnings
import warnings

# warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.INFO)

from typing import Optional, Tuple, Any

from samna.dynapse2 import *
import samna

# Utils
import numpy as np
from tqdm import tqdm
import time

# Rockpool Imports
from rockpool.timeseries import TSEvent
from rockpool.nn.modules.jax import LinearJax
from rockpool.nn.modules.jax import LIFJax
from rockpool.nn.combinators import Sequential

# DynapSE
from rockpool.devices.dynapse.dynapsim import DynapSim
from rockpool.devices.dynapse.utils.figure import Figure
from rockpool.devices.dynapse.utils.spike_input import poisson_spike_train
from rockpool.nn.modules.module import ModuleBase
# from rockpool.devices.dynapse.ref.weights_old import WeightParameters
from rockpool.devices.dynapse.quantize.weight_handler import WeightHandler

## Dynap-SE2
from rockpool.devices.dynapse.config.simconfig import DynapSimConfig
from rockpool.devices.dynapse.samna_alias import dynapse2 as ds2
from rockpool.devices.dynapse.interface.utils import find_dynapse_boards, configure_dynapse2_fpga, disconnect, raster_to_aer, capture_events_from_device
from rockpool.devices.dynapse.interface.samna import DynapseSamna

from rockpool.devices.dynapse.samna_alias.dynapse2 import Dynapse2Configuration
# - Plotting and config
import matplotlib.pyplot as plt

# %matplotlib inline
plt.rcParams["figure.figsize"] = [5.6, 2.8]
plt.rcParams["figure.dpi"] = 1200
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 12


/Users/ugurcancakal/rockpool_xylosim/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## Connect to device
se2_devices = find_dynapse_boards()

INFO:root: Total of 1 DYNAP-SE2 board(s) found with serial numbers : ['']


In [3]:
found_se2 = len(se2_devices) > 0

if not found_se2:
    raise IOError('This tutorial requires a connected Dynap-SE2 Stack Board HDK to demonstrate.')
else:
    # board = configure_dynapse2_fpga(se2_devices[0])
    se2_stack = se2_devices[0]

In [4]:
se2 = DynapseSamna((60,2), se2_stack)
myConfig = se2.config

INFO:root:DYNAP-SE2 Stack is connected, configured and ready for operation!


In [5]:
myConfig

In [6]:
se2_conf = Dynapse2Configuration.from_samna(myConfig)

In [7]:
myConfig.to_json() == se2_conf.to_json()

True

In [8]:
myConfig.chips[0].cores[0].neurons[0].destinations[0] == se2_conf.chips[0].cores[0].neurons[0].destinations[0].to_samna()

True

In [9]:
se2_conf.chips[0].cores[0].neurons[0].destinations[0].to_samna()

In [10]:
se2_conf.chips[0].cores[0].neurons[0].destinations[0].to_json()

'{\n    "value0": {\n        "core": {\n            "value0": false,\n            "value1": false,\n            "value2": false,\n            "value3": false\n        },\n        "xHop": 0,\n        "yHop": 0,\n        "tag": 0\n    }\n}'

In [11]:
# print(myConfig.chips[0].cores[0].neurons[0].synapses[0].to_json())

In [12]:
# myConfig.chips[0].cores[0].neurons[0].destinations[0]

In [13]:
# print(myConfig.chips[0].cores[0].to_json())

In [14]:
samna.dynapse2.NormalGridEvent()

In [15]:
from rockpool.devices.dynapse.samna_alias.dynapse2 import NormalGridEvent, Dynapse2Destination

In [16]:
core = [1, 0 , 0, 0]
x_hop = 1
y_hop = 1
tag = 1024
event_time = 1.2
dt_fpga = 1e-6

event = NormalGridEvent(
    event=Dynapse2Destination(core, x_hop, y_hop, tag),
    timestamp=int(event_time / dt_fpga),
)

In [17]:
print(event.to_json() == event.to_samna().to_json())

True


In [18]:
print(event.to_samna().to_json())

{
    "value0": {
        "event": {
            "core": {
                "value0": true,
                "value1": false,
                "value2": false,
                "value3": false
            },
            "xHop": 1,
            "yHop": 1,
            "tag": 1024
        },
        "timestamp": 1200000
    }
}


In [19]:
se2_conf.to_json() == myConfig.to_json()

True

In [20]:
j_se2_conf = se2_conf.to_samna().to_json()

with open("jse2.json", "w") as f:
    f.write(j_se2_conf)

with open("mconfig.json", "w") as f:
    f.write(myConfig.to_json())

In [ ]:
se2_conf.save()